In [ ]:
# %%
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
from folium.plugins import HeatMap
from shapely.geometry import Point, LineString, Polygon
import shapely
import momepy as mp 
import esda
import seaborn as sns
sns.set_theme()


# %%
# ### convert polygon to geodataframe
# gdf = gpd.GeoDataFrame(index=[0], crs='EPSG:3857', geometry=[poly])
# gdf = gdf.to_crs('EPSG:4326'); gdf = gdf['geometry'][0]


# %%
# oslo = ox.graph.graph_from_polygon(gdf, network_type='all', simplify=True)


# %%
lat, lon = 55.6867243, 12.5700724


# %%
def get_city_graph(lat, lon, dist, polygonize=False, plot = False):
    g = ox.graph_from_point((lat, lon), dist=dist, network_type='bike', simplify=True, retain_all=False)
    gdf = mp.nx_to_gdf(g)
    edges = gdf[1].to_crs('EPSG:3857')
    if polygonize:
        linestrings = edges.geometry # our geopandas.GeoSeries of linestrings representing street network
        collection = shapely.GeometryCollection(linestrings.array)  # combine to a single object
        noded = shapely.node(collection)  # add missing nodes
        polygonized = shapely.polygonize(noded.geoms)  # polygonize based on an array of noded parts
        polygons = gpd.GeoSeries(polygonized.geoms)  # create a GeoSeries from parts
        return g, edges, polygons
    if plot:
        ### plot the graph
        fig, ax = ox.plot_graph(g, node_size=0, edge_linewidth=0.5, show=False, close=False)
        edges.plot(ax=ax, linewidth=1, edgecolor='black')
        plt.show()
    return g, edges


# %%
g, edges = get_city_graph(lat, lon, 10000)


# # %%
# ### plot the graph
fig, ax = ox.plot_graph(g, node_size=0, edge_linewidth=0.5, show=False, close=False)
edges.plot(ax=ax, linewidth=1, edgecolor='black')


# %%
import nx_parallel as nxp
import networkx as nx


# %%

# ebc = nxp.edge_betweenness_centrality(g, normalized=False, weight='length')


# %%
### carry weights over to line graph
H = nx.line_graph(g)
H.add_nodes_from((node, g.edges[node]) for node in H)


In [ ]:
list(H.edges(data=True))[0]
# len(list(H.nodes(data=True)))


In [4]:
# %%
for s, t, v in H.edges:
    H.edges[(s, t, v)]['weight'] = g.edges[s]['length'] + g.edges[t]['length']


In [5]:
ebc = dict(nxp.all_pairs_dijkstra_path(H, weight='weight', cutoff=1000))
def calc_bc(shortest_paths, graph):
    bc = {i : 0 for i in graph.nodes}
    for node in graph.nodes:
        for other_node in shortest_paths[node].keys():
            path = shortest_paths[node][other_node]
            for node_visited in path:
                bc[node_visited] += 1
    for node in bc.keys():
        bc[node] /= len(graph.nodes)
    return bc
bc = calc_bc(ebc, H)


In [6]:
bc = {k: v for k, v in sorted(bc.items(), key=lambda item: item[1], reverse=True)}


In [7]:
### color edges in g by bc
bc2 = {}
for x, y, z in bc:
    bc2[(x, y)] = bc[(x, y, z)]

for (s, t), value in bc2.items():
    for i in range(len(g[s][t])):
        g[s][t][i]['bc'] = value


In [ ]:
### scale bc
cph = mp.nx_to_gdf(g)
cph_edges = cph[1].to_crs('EPSG:3857')
cph_edges['bc'] = cph_edges['bc'] / cph_edges['bc'].max()
cph_edges['bc'] = np.log10(cph_edges['bc'])


In [ ]:
# fig, ax = ox.plot_graph(g, node_size=0, edge_linewidth=0.5, show=False, close=False)
cph_edges.plot(linewidth=1, column='bc', cmap='turbo', legend=True, figsize=(10, 10))
### drop axes
plt.axis('off')
plt.show()


In [ ]:
ebc = dict(nx.all_pairs_dijkstra_path(H, weight='weight', cutoff=500))
def calc_bc(shortest_paths, graph):
    bc = {i : 0 for i in graph.nodes}
    for node in graph.nodes:
        for other_node in shortest_paths[node].keys():
            path = shortest_paths[node][other_node]
            for node_visited in path:
                bc[node_visited] += 1
    for node in bc.keys():
        bc[node] /= len(graph.nodes)
    return bc
bc = calc_bc(ebc, H)

### color edges in g by bc
bc2 = {}
for x, y, z in bc:
    bc2[(x, y)] = bc[(x, y, z)]

for (s, t), value in bc2.items():
    for i in range(len(g[s][t])):
        g[s][t][i]['bc'] = value

### scale bc
cph = mp.nx_to_gdf(g)
cph_edges = cph[1].to_crs('EPSG:3857')
cph_edges['bc'] = cph_edges['bc'] / cph_edges['bc'].max()
cph_edges['bc'] = np.log10(cph_edges['bc'])
# fig, ax = ox.plot_graph(g, node_size=0, edge_linewidth=0.5, show=False, close=False)
cph_edges.plot(linewidth=1, column='bc', cmap='turbo', legend=True, figsize=(10, 10))
### drop axes
plt.axis('off')
plt.show()
